In [1]:
import pandas as pd
import numpy as np
import torch
import os
import sys
from tqdm import tqdm, trange
import time
import copy
sys.path.append("../../")

# print("Current working directory:", os.getcwd())

from biked_commons.design_evaluation.design_evaluation import *
from biked_commons.design_evaluation.scoring import *
from biked_commons.data_loading import data_loading

# os.chdir("src")
# print("Current working directory:", os.getcwd())

import argparse
import torch
import numpy as np
from matplotlib import pyplot as plt
from importlib import invalidate_caches
invalidate_caches()

from biked_commons.benchmark_models.libmoon.solver.gradient import MGDASolver, GradAggSolver, EPOSolver, MOOSVGDSolver, GradHVSolver, PMTLSolver
from biked_commons.benchmark_models.libmoon.util_global.constant import problem_dict
from biked_commons.benchmark_models.libmoon.util_global.weight_factor.funs import uniform_pref
from biked_commons.benchmark_models.libmoon.visulization.view_res import vedio_res
from biked_commons.benchmark_models.libmoon.problem.mop import mop
from biked_commons.benchmark_models import benchmarking_utils

import sys
print(sys.version)

c:\Users\Lyler\mambaforge\envs\torch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


3.12.8 | packaged by conda-forge | (main, Dec  5 2024, 14:06:27) [MSC v.1942 64 bit (AMD64)]


In [2]:
data = data_loading.load_bike_bench_train()
save_results = False

In [3]:
device = torch.device("cpu")
# "cuda" if torch.cuda.is_available() else

In [4]:
class MOOProblem(mop):
    def __init__(self, data_sample_df, condition=None, device=None):
        self.evaluator, self.requirement_names, requirement_types = construct_tensor_evaluator(
            get_standard_evaluations(device), data.columns, device=device
        )

        self.isobjective = torch.tensor(requirement_types) == 1
        self.objective_names = np.array(self.requirement_names)[self.isobjective.numpy()]
        self.ref_point = get_ref_point(self.evaluator, self.objective_names, self.isobjective)

        data_sample = data_sample_df.to_numpy()
        n_var = data_sample.shape[1]
        lbound = np.min(data_sample, axis=0)
        ubound = np.max(data_sample, axis=0)

        super().__init__(
            n_var=n_var,
            n_obj=self.isobjective.sum().item(),
            lbound=lbound,
            ubound=ubound
        )

        self.problem_name = 'MOO'
        self.condition = condition

    def eval_fn(self, x, condition):
        score = self.evaluator(x, condition)
        objective_scores = score[:, self.isobjective]
        constraint_scores = score[:, ~self.isobjective]
        constraint_violations = torch.clamp(constraint_scores, min=0.0)
        penalty = constraint_violations.sum(dim=1, keepdim=True)
        penalized_objectives = objective_scores + 1e3 * penalty
        return penalized_objectives

    def _evaluate_torch(self, x):
        return self.eval_fn(x, self.condition)

In [5]:
def test(problem, args, x0, prefs, solver):
    print("=" * 50)
    print(f"Running test with solver: {solver.__class__.__name__}")
    start_time = time.time()
    
    res = solver.solve(
        problem,
        x=x0,
        prefs=prefs,
        args=args,
        ref_point=problem.ref_point
    )

    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Time completed: {elapsed_time:.2f} seconds")

    x0_np = x0.cpu().numpy() if isinstance(x0, torch.Tensor) else x0
    res_x_np = res['x'].cpu().numpy() if isinstance(res['x'], torch.Tensor) else res['x']

    assert x0_np.shape == res_x_np.shape, "Shape mismatch between x0 and result"

    num_affected = sum(
        np.count_nonzero(res_x_np[i] != x0_np[i])
        for i in range(x0_np.shape[0])
    )
    avg_affected = num_affected / x0.shape[0]
    print(f"Average Number of Parameters Affected: {avg_affected:.2f}")
    print("=" * 50)
    return res

In [6]:
def run_solver(cond_idx, solver_name="agg", device=device):
    condition = benchmarking_utils.get_condition_by_idx(cond_idx)
    parser = argparse.ArgumentParser(description='example')
    parser.add_argument('--n-partition', type=int, default=10)
    parser.add_argument('--agg', type=str, default='tche')
    parser.add_argument('--solver', type=str, default='agg')
    parser.add_argument('--iter', type=int, default=1000)
    parser.add_argument('--step-size', type=float, default=1e-2)
    parser.add_argument('--tol', type=float, default=1e-6)

    args = parser.parse_args(args=[
        '--n-partition', '2',
        '--agg', 'tche',
        '--solver', 'agg',
        '--iter', '10',
        # '--step-size', '0.1',
        '--tol', '1e-6'
    ])
    args.n_var = data.shape[1]

    problem = MOOProblem(data_sample_df=data, condition=condition, device=device)

    args.n_obj = problem.n_obj
    # prefs = uniform_pref(args.n_partition, args.n_obj, clip_eps=1e-2)
    prefs = torch.tensor(
        uniform_pref(args.n_partition, args.n_obj, clip_eps=1e-2),
        dtype=torch.float32,
        device=device 
    )
    args.n_prob = len(prefs)

    # x0 = torch.tensor(data.iloc[0].to_numpy(), dtype=torch.float32).repeat(args.n_prob, 1)
    x0 = torch.tensor(
        data.sample(n=args.n_prob, replace=True).to_numpy(), 
        dtype=torch.float32,
        device=device
    )
    # Logging setup info
    print(f"Number of Objectives: {args.n_obj}")
    print(f"Number of Preferences: {args.n_prob}")
    print(f"Number of Constraints: {(~problem.isobjective).sum().item()}")
    print(f"x0 shape: {x0.shape}")
    # print(f"Condition: {condition}")
    print(f"Requirement Names: {problem.requirement_names}")
    print(f"Ref Point: {problem.ref_point}")

    print(f"Testing solver: {solver_name}")
    
    if solver_name == 'mgda':
        args.step_size = 0.1
        num_trials = 1
        solver = MGDASolver(args.step_size, args.iter, args.tol)  # Small grads breaks it
    elif solver_name == 'Agg-LS':
        args.step_size = 0.1
        args.iter = 1 #1000 for full run
        num_trials = 1
        solver = GradAggSolver(args.step_size, args.iter, args.tol, device)  # Works well
    elif solver_name == 'EPO':
        args.step_size = 0.1
        num_trials = 1
        solver = EPOSolver(args.step_size, args.iter, args.tol) # Sensitive to NaN values (aero breaks it)
    else:
        raise ValueError(f"Solver '{solver_name}' not supported.")
    
    all_results = []
    for i in range(num_trials):
        res = test(
            problem,
            copy.deepcopy(args),
            x0.clone(),
            prefs.clone(),
            solver
        )
        all_results.append(torch.tensor(res['x'], dtype=torch.float32, device=device))
    all_results = torch.concat(all_results, dim=0)
    benchmarking_utils.evaluate_uncond(all_results, solver_name, cond_idx, data.columns, device, save=save_results)
    return problem, res

# for i in range(10):
#     for solver in ['Agg-LS', 'EPO']:
#         problem, res = run_solver(cond_idx=i, solver_name=solver, device = device)
i = 0
solver = 'Agg-LS'
# solver = 'EPO'
problem, res = run_solver(cond_idx=i, solver_name=solver, device = device)

Number of Objectives: 10
Number of Preferences: 55
Number of Constraints: 15
x0 shape: torch.Size([55, 90])
Requirement Names: ['Usability Score - 0 to 1', 'Drag Force', 'Knee Angle Error', 'Hip Angle Error', 'Arm Angle Error', 'Cosine Distance to Embedding', 'Mass', 'Planar Compliance', 'Transverse Compliance', 'Eccentric Compliance', 'Planar Safety Factor', 'Eccentric Safety Factor', 'Saddle height too small', 'Seat post too short', 'Head tube lower extension too great', 'Head tube length too great', 'Certain parameters must be positive', 'Chain stay should be greater than wheel radius', 'Chain stay should be greater than BB', 'Seat stay should be greater than wheel radius', 'Down tube must reach head tube', "The pedal shouldn't intersect the front wheel", "The crank shouldn't hit the ground when it is in its lower position", 'RGB value should be less than 255', 'Predicted Frame Validity']
Ref Point: [1.0000000e+00 2.9636734e+01 1.9729372e+02 8.0775903e+02 8.4349000e+02
 5.4179370e-0

100%|██████████| 1/1 [00:00<00:00,  3.57it/s]


Time completed: 0.29 seconds
Average Number of Parameters Affected: 72.67
